# Simple diabete regression 

## Imports

In [1]:
import numpy as np

from sklearn.datasets import load_diabetes

from neuralnetlib.preprocessing import MinMaxScaler, StandardScaler
from neuralnetlib.activations import Linear, LeakyReLU
from neuralnetlib.layers import Input, Dense, Activation
from neuralnetlib.losses import MeanSquaredError, MeanAbsoluteError
from neuralnetlib.models import Sequential
from neuralnetlib.optimizers import Adam
from neuralnetlib.utils import train_test_split

## 1. Loading a dataset (in this case, the diabetes dataset)

In [2]:
x, y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## 2. Preprocessing

In [3]:
scaler_x = MinMaxScaler()
x_train = scaler_x.fit_transform(x_train)
x_test = scaler_x.transform(x_test)
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

## 3. Model definition

In [4]:
input_neurons = x_train.shape[1:][0]
num_hidden_layers = 2
hidden_neurons = 2
output_neurons = 1

model = Sequential()
model.add(Input(input_neurons))
model.add(Dense(hidden_neurons, weights_init='he_normal', random_state=42))
model.add(Activation(LeakyReLU()))

for _ in range(num_hidden_layers - 1):
    model.add(Dense(hidden_neurons, weights_init='he_normal', random_state=42))
    model.add(Activation(LeakyReLU()))

model.add(Dense(output_neurons, random_state=42))
model.add(Activation(Linear()))

## 4. Model compilation

In [5]:
model.compile(loss_function=MeanSquaredError(), optimizer=Adam(learning_rate=0.001))

model.summary()

Sequential(gradient_clip_threshold=5.0, enable_padding=False, padding_size=32, random_state=1733515038822283600)
-------------------------------------------------
Layer 1: Input(input_shape=(10,))
Layer 2: Dense(units=2)
Layer 3: Activation(LeakyReLU)
Layer 4: Dense(units=2)
Layer 5: Activation(LeakyReLU)
Layer 6: Dense(units=1)
Layer 7: Activation(Linear)
-------------------------------------------------
Loss function: MeanSquaredError
Optimizer: Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clip_norm=None, clip_value=None)
-------------------------------------------------



## 5. Model training

In [6]:
model.fit(x_train, y_train, epochs=10, batch_size=32, random_state=42)

[==============================] 100% Epoch 1/10 - 0.01s - loss: 1.2543         
[==============================] 100% Epoch 2/10 - 0.01s - loss: 1.2482         
[==============================] 100% Epoch 3/10 - 0.01s - loss: 1.2422         
[==============================] 100% Epoch 4/10 - 0.01s - loss: 1.2366         
[==============================] 100% Epoch 5/10 - 0.01s - loss: 1.2320         
[==============================] 100% Epoch 6/10 - 0.01s - loss: 1.2275         
[==============================] 100% Epoch 7/10 - 0.01s - loss: 1.2231         
[==============================] 100% Epoch 8/10 - 0.01s - loss: 1.2183         
[==============================] 100% Epoch 9/10 - 0.01s - loss: 1.2134         
[==============================] 100% Epoch 10/10 - 0.01s - loss: 1.2083        



## 6. Model evaluation

In [7]:
loss, _ = model.evaluate(x_test, y_test)
print(f'Test loss: {loss}', "function=" + str(model.loss_function))

Test loss: 1.1136541600695817 function=MeanSquaredError


## 7. Model prediction

In [8]:
y_pred = model.predict(x_test)
print("MAE: ", MeanAbsoluteError()(y_test, y_pred))

MAE:  0.8748635782918366


## 8. Getting original MAE (without normalization from StandardScaler)

In [9]:
y_pred_scaled = model.predict(x_test)

y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

mae_original = np.mean(np.abs(y_test_original - y_pred))
print(f'MAE (original): {mae_original}')

MAE (original): 65.9770599527072
